In [1]:
import glob
import os
import scipy.sparse as sps
import numpy as np
import pandas as pd
from sklearn import utils


In [2]:
import random
import re
import os
import shutil
import sys
import json
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from collections import Counter


In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [4]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [6]:
from skmultilearn.problem_transform import LabelPowerset # pip3 install scikit-multilearn
from imblearn.over_sampling import RandomOverSampler

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
Dataset_ori = pd.read_csv('/Users/namrata/Documents/PSU/Thesis_research/Cybersecurity_framework/Data/OPP-115/data_to_gitlab/Cleaned_multilabel_dataset.txt',sep='\t')
del Dataset_ori['Other']
Dataset_ori.head(2)

,website,alltext,segment ID,Data Retention,Data Security,Do Not Track,First Party Collection/Use,International and Specific Audiences,Introductory/Generic,Policy Change,Practice not covered,Privacy contact information,Third Party Sharing/Collection,"User Access, Edit and Deletion",User Choice/Control
0,1017_sci-news.com,- details of your visits to our site including...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1017_sci-news.com,"- if you contact us, we may keep a record of t...",2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
Dataset_ori = Dataset_ori.reset_index(drop=True)

In [11]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [13]:
Dataset_ori['alltext'] = Dataset_ori['alltext'].apply(lambda x:clean_text(x))

In [70]:
# Dataset_ori.head(10)

In [14]:
# from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score


In [15]:
labeled_questions=[]
for i in range(len(Dataset_ori)):
    labeled_questions.append(TaggedDocument(Dataset_ori['alltext'].iloc[i].split(), Dataset_ori[Dataset_ori.index == i].index))

In [14]:
# labeled_questions

In [16]:
labeled_questions[2345]

TaggedDocument(words=['we', 'use', 'third', 'party', 'advertising', 'companies', 'to', 'serve', 'ads', 'when', 'you', 'visit', 'websites', 'operated', 'by', 'the', 'news', 'post', 'these', 'companies', 'may', 'use', 'information', 'not', 'including', 'your', 'name', 'address', 'email', 'address', 'or', 'telephone', 'number', 'about', 'your', 'visits', 'to', 'this', 'and', 'other', 'web', 'sites', 'in', 'order', 'to', 'provide', 'advertisements', 'about', 'goods', 'and', 'services', 'of', 'interest', 'to', 'you', 'if', 'you', 'would', 'like', 'more', 'information', 'about', 'this', 'practice', 'and', 'to', 'know', 'your', 'choices', 'about', 'not', 'having', 'this', 'information', 'used', 'by', 'these', 'companies', 'visit', 'http', 'www', 'networkadvertising', 'org'], tags=Int64Index([2345], dtype='int64'))

In [17]:
model = Doc2Vec(dm = 0, min_count=1, window=10, size=300,alpha=0.01)
model.build_vocab(labeled_questions)

In [76]:
model.corpus_count

3574

In [18]:
# %%time
for epoch in range(50):
    print('Epoch {}'.format(epoch))
    model.train(utils.shuffle([x for x in labeled_questions]), total_examples=len(labeled_questions), epochs=1)
#     model.alpha -= 0.002
#     model.min_alpha = model.alpha

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49


In [35]:
# for epoch in range(50):
#     model.train(labeled_questions,epochs=model.iter,total_examples=model.corpus_count)
#     print("Epoch #{} is complete.".format(epoch+1))

In [19]:
Dataset_ori["Id"] = Dataset_ori['website'] + Dataset_ori['segment ID'].astype('str')
labels_order = Dataset_ori.columns[3:-1]
all_websites = Dataset_ori['website'].unique()
web_dict = Counter(all_websites)

selection_per_loop = dict()

fraction =1 

In [20]:
folds = 10

lp = LabelPowerset()
ros = RandomOverSampler(random_state=420)


for count in range(folds):
# count=0
    print('Fold: ',count)
    itemminValue = min(web_dict.items(), key=lambda x: x[1])

    min_val = itemminValue[1]
    minimum_webs = list()
    # print(min_val)
    for key, value in web_dict.items():
        if value == min_val:
            minimum_webs.append(key)
    random.shuffle(minimum_webs)

    selected_test_webs = minimum_webs[0:40]

    while len(selected_test_webs)<40:
        min_val+=1
        minimum_webs = list()
        for key, value in web_dict.items():
            if value == min_val:
                minimum_webs.append(key)
        random.shuffle(minimum_webs)
        selected_test_webs = list(set(selected_test_webs)|set(minimum_webs[0:40-len(selected_test_webs)]))
        # print('final len of test sites',len(selected_test_webs))

    for tw in selected_test_webs:
        web_dict[tw] += 1
    selection_per_loop[count] = selected_test_webs

    train_df = Dataset_ori[~Dataset_ori['website'].isin(selected_test_webs)]
    test_df = Dataset_ori[Dataset_ori['website'].isin(selected_test_webs)]

    train_df=train_df.sample(frac=fraction)
    test_df=test_df.sample(frac=fraction)

    train_text_ori = train_df['alltext'].tolist()
    # train_text_ori = [' '.join(t.split()) for t in train_text_ori]
    # train_text_ori = np.array(train_text_ori, dtype=object)[:, np.newaxis]
    train_label_ori = train_df.values[:,3:-1].astype(int)

    test_text = test_df['alltext'].tolist()
    # test_text = [' '.join(t.split()) for t in test_text]
    # test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    test_label = test_df.values[:,3:-1].astype(int)
    

    X1 = []
    for i in range(len(train_text_ori)):
        fullText = clean_text(train_text_ori[i])
        vec_inp = fullText.strip().split()
        vector = np.array([x for x in model.infer_vector(vec_inp,steps=100,alpha=0.01)])
        X1.append(vector)
    train_X = np.array(X1)

    X2 = []
    for i in range(len(test_text)):
        fullText = clean_text(test_text[i])
        vec_inp = fullText.strip().split()
        vector = np.array([x for x in model.infer_vector(vec_inp,steps=100,alpha=0.01)])
        X2.append(vector)
    test_X = np.array(X2)

    # print(test_X.shape)

    train_X_norm = min_max_scaler.fit_transform(train_X)
    test_X_norm = min_max_scaler.fit_transform(test_X)

    print('Before upsampling: ',train_X_norm.shape,train_label_ori.shape,test_X_norm.shape,test_label.shape)

    yt = lp.transform(train_label_ori)
    train_X_norm_up, y_resampled = ros.fit_sample(train_X_norm, yt)
    # train_text = train_text_ori
    # train_label = train_label_ori
    train_label = lp.inverse_transform(y_resampled).toarray()

    print('After Up-sampling',train_X_norm_up.shape,train_label.shape,test_X_norm.shape,test_label.shape)
    
    
    NB_pipeline = Pipeline([
                    #('tfidf', TfidfVectorizer(stop_words=stop_words)),
                    ('clf', OneVsRestClassifier(MultinomialNB(
                        fit_prior=True, class_prior=None))),
                ])        

    all_pred_res = []
    for category in range(train_label.shape[1]):
        NB_pipeline.fit(train_X_norm_up, train_label[:,category].reshape((train_label.shape[0], 1)))
        preds = NB_pipeline.predict(test_X_norm)
        all_pred_res.append(preds)

    # print(len(all_pred_res))
    pred_binary = np.asarray(all_pred_res).T

    # print(pred_binary.shape)

    result_scores = pd.DataFrame()
    avg_f1 = []
    for category in range(train_label.shape[1]):
        label_name = labels_order[category]

    #     lr_precision, lr_recall, _ = precision_recall_curve(test_label[:,category].astype('float'), pre_save_preds[:,category])
        lr_f1 = f1_score(test_label[:,category].astype(int), pred_binary[:,category])#, auc(lr_recall, lr_precision)

        if sum(test_label[:,category])==0 and sum(pred_binary[:,category])==0:
            tp=0
            fp=0
            tn=len(pred_binary[:,category])
            fn=0
        else:
            tn, fp, fn, tp = confusion_matrix(test_label[:,category].astype(int), pred_binary[:,category]).ravel()

        if tp+fp==0:
            precision = 0
        else:
            precision = float(tp)/float(tp+fp)

        if tp+fn==0:
            recall = 0
        else:
            recall = float(tp)/float(tp+fn)

        avg_f1.append(lr_f1)
        df = pd.DataFrame([{'Fold':count,'category':label_name,'tp':tp,'fp':fp,'tn':tn,'fn':fn,'recall':recall,'precision':precision,'f1':lr_f1}])

        result_scores = result_scores.append(df)
#     print(result_scores)
    print('Average resulting F-1 Score in fold',np.mean(avg_f1))
#     if count==0:
#         result_scores.to_csv('/Users/namrata/Documents/PSU/Thesis_research/Cybersecurity_framework/Exp_res/Doc2Vec_Classi/OPP_115/Doc2Vec_OPP_res_selftrained_upsampled.csv',header=True,mode='a',sep='\t')
#     else:
#         result_scores.to_csv('/Users/namrata/Documents/PSU/Thesis_research/Cybersecurity_framework/Exp_res/Doc2Vec_Classi/OPP_115/Doc2Vec_OPP_res_selftrained_upsampled.csv',header=False,mode='a',sep='\t')

    count+=1 

Fold:  0
Before upsampling:  (2271, 300) (2271, 12) (1303, 300) (1303, 12)
After Up-sampling (41472, 300) (41472, 12) (1303, 300) (1303, 12)
Average resulting F-1 Score in fold 0.1862504963477536
Fold:  1
Before upsampling:  (2350, 300) (2350, 12) (1224, 300) (1224, 12)
After Up-sampling (46080, 300) (46080, 12) (1224, 300) (1224, 12)
Average resulting F-1 Score in fold 0.14891595017463197
Fold:  2
Before upsampling:  (2413, 300) (2413, 12) (1161, 300) (1161, 12)
After Up-sampling (51408, 300) (51408, 12) (1161, 300) (1161, 12)
Average resulting F-1 Score in fold 0.19945457326258506
Fold:  3
Before upsampling:  (2237, 300) (2237, 12) (1337, 300) (1337, 12)
After Up-sampling (40560, 300) (40560, 12) (1337, 300) (1337, 12)
Average resulting F-1 Score in fold 0.1725251461674662
Fold:  4
Before upsampling:  (2425, 300) (2425, 12) (1149, 300) (1149, 12)
After Up-sampling (50301, 300) (50301, 12) (1149, 300) (1149, 12)
Average resulting F-1 Score in fold 0.1415503283490035
Fold:  5
Before up

In [24]:
train_text.shape

(48590, 300)